In [11]:
#from coordinate_descent import SparseCoordinateDescentSVM
from CDSVM_Michal_archive import CoordinateDescentSVM
import data_loader
import time

In [2]:
X_tr,X_test, y_tr, y_test = data_loader.load_dataset('../data/paper_data/news20.binary')

In [3]:
print("Shape:", X_tr.shape)
print("Sparsity:", 100 * (1 - X_tr.nnz / (X_tr.shape[0] * X_tr.shape[1])), "%")


Shape: (15996, 1355192)
Sparsity: 99.96638617384322 %


In [37]:
from  sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_tr,y_tr)

LinearSVC()

In [3]:
model = SparseCoordinateDescentSVM()

In [4]:
model.fit(X_tr,y_tr)

0
0
1
2
3
4
5
6
7
8


KeyboardInterrupt: 

In [3]:
model = CoordinateDescentSVM()
model.fit(X_tr,y_tr)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [10]:
print(X_tr.shape)
print(np.ones((X_tr.shape[0], 1)).shape)
X_ext = np.hstack([X_tr, np.ones((X_tr.shape[0], 1))])

(15996, 1355192)
(15996, 1)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [15]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

# Załaduj dane w formacie sparse

# Linear SVM (działa dobrze na sparse)
clf = LinearSVC(dual=False)  # dual=False działa lepiej, gdy n_samples > n_features
clf.fit(X_tr, y_tr)

#print("Test accuracy:", clf.score(X_test, y_test))


LinearSVC(dual=False)

In [6]:
X_tr.shape

(15996, 1355192)

# Tutaj są moje zmiany!

In [89]:
import numpy as np
import time
from scipy.sparse import issparse, csc_matrix


class SparseCoordinateDescentSVM_2:
    def __init__(self, C=1.0, max_iter=1000, tol=1e-8, sigma=0.01, beta=0.5, verbose=True):
        self.C = C
        self.max_iter = max_iter
        self.tol = tol
        self.sigma = sigma
        self.beta = beta
        self.verbose = verbose
        self.w = None
        self.lambdas = {}

    def _d_prime_i_0(self, X, y, i):
        margins = 1 - y * (X @ self.w)
        active = margins > 0
        bj = margins[active]
        y_active = y[active]
        xji = X[active, i]

        if issparse(xji):
            gradient_sum = xji.multiply(y_active * bj).sum()
        else:
            gradient_sum = np.sum(y_active * xji * bj)

        return self.w[i] - 2 * self.C * gradient_sum

    def _d_double_prime_i_0(self, X, y, i):
        margins = 1 - y * (X @ self.w)
        active = margins > 0
        xji = X[active, i]

        if issparse(xji):
            hessian_sum = xji.multiply(xji).sum()
        else:
            hessian_sum = np.sum(xji ** 2)

        return 1 + 2 * self.C * hessian_sum

    def _newton_direction(self, X, y, i):
        numerator = self._d_prime_i_0(X, y, i)
        denominator = self._d_double_prime_i_0(X, y, i)
        return -numerator / denominator if denominator != 0 else 0.0

    def _d_i_z(self, X, y, i, z):
        ei = np.zeros(X.shape[1])
        ei[i] = 1
        w_new = self.w + z * ei
        margins = 1 - y * (X @ w_new)
        active = margins > 0
        loss_term = np.sum(margins[active] ** 2)
        return 0.5 * np.dot(w_new, w_new) + self.C * loss_term

    def _compute_constant_lambda(self, X, y, i):
        dii = self._d_double_prime_i_0(X, y, i)

        Xi_col = X[:, i]
        if issparse(Xi_col):
            Xi_squared_sum = Xi_col.multiply(Xi_col).sum()
        else:
            Xi_squared_sum = np.sum(Xi_col ** 2)

        Hi = 1 + 2 * self.C * Xi_squared_sum
        return dii / (0.5 * Hi + self.sigma)

    def _compute_lambda(self, X, y, i, d):
        lambda_bar = self.lambdas[i]
        if abs(d) < 1e-12:
            return 0.0
        if 1.0 <= lambda_bar:
            return 1.0

        # Otherwise, perform line search
        D0 = self._d_i_z(X, y, i, 0)
        k = 0
        while True:
            lam = self.beta ** k
            z = lam * d
            Dz = self._d_i_z(X, y, i, z)
            if Dz - D0 <= -self.sigma * (z ** 2):
                return lam
            k += 1
            if k > 20:  # Prevent infinite loop
                return lam

    def _coordinate_update(self, X, y, i):
        d = self._newton_direction(X, y, i)
        if abs(d) < 1e-12:
            return
        lam = 1.0
        if i in self.lambdas:
            if 1.0 > self.lambdas[i]:  # Only check line search if needed
                lam = self._compute_lambda(X, y, i, d)
        else:
            self.lambdas[i] = self._compute_constant_lambda(X, y, i)
            if 1.0 > self.lambdas[i]:
                lam = self._compute_lambda(X, y, i, d)
        self.w[i] += lam * d


    def fit(self, X, y):
        if not issparse(X) or not isinstance(X, csc_matrix):
            raise ValueError("X must be a CSC (Compressed Sparse Column) matrix")
    
        self.w = np.zeros(X.shape[1])
    
        for iteration in range(self.max_iter):
            w_old = self.w.copy()
            i = np.random.randint(0, X.shape[1])
            if i not in self.lambdas:
                self.lambdas[i] = self._compute_constant_lambda(X, y, i)
            self._coordinate_update(X, y, i)
    
            delta = np.linalg.norm(self.w - w_old)
            train_error = 1.0 - self.score(X, y)
    
            if self.verbose:
                print(f"Iter {iteration}, Δw = {delta:.2e}, train error = {train_error:.4f}")
    
            if delta < self.tol:
                if self.verbose:
                    print(f"Converged at iteration {iteration}")
                break
    
        return self


    def predict(self, X):
        return np.sign(X @ self.w)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)


In [92]:
model = SparseCoordinateDescentSVM_2(C = 4)


In [69]:
def load_svm_file(file_path, zero_based=True):
    labels = []
    rows = []
    cols = []
    data = []

    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            parts = line.strip().split()
            labels.append(float(parts[0]))

            for feat in parts[1:]:
                idx, val = feat.split(':')
                idx = int(idx) - (0 if zero_based else 1)
                rows.append(i)
                cols.append(idx)
                data.append(float(val))

    # Jawna konwersja do CSC
    from scipy.sparse import coo_matrix
    X = coo_matrix((data, (rows, cols))).tocsc()
    y = np.array(labels)

    return X, y
X,y =load_svm_file('../data/paper_data/news20.binary')

In [93]:
model.fit(X,y)

Iter 0, Δw = 1.46e+00, train error = 1.0000
Iter 1, Δw = 1.14e+00, train error = 1.0000
Iter 2, Δw = 4.16e+00, train error = 0.9995
Iter 3, Δw = 2.46e-01, train error = 0.9994
Iter 4, Δw = 4.67e-01, train error = 0.9993
Iter 5, Δw = 3.80e-01, train error = 0.9991
Iter 6, Δw = 3.23e-01, train error = 0.9991
Iter 7, Δw = 9.09e-01, train error = 0.9987
Iter 8, Δw = 2.81e-01, train error = 0.9987
Iter 9, Δw = 3.44e-01, train error = 0.9987
Iter 10, Δw = 6.71e-02, train error = 0.9987
Iter 11, Δw = 2.27e-01, train error = 0.9987
Iter 12, Δw = 5.05e-02, train error = 0.9986
Iter 13, Δw = 8.24e-02, train error = 0.9985
Iter 14, Δw = 7.77e-03, train error = 0.9985
Iter 15, Δw = 3.00e-02, train error = 0.9984
Iter 16, Δw = 2.38e-02, train error = 0.9984
Iter 17, Δw = 9.00e-03, train error = 0.9984
Iter 18, Δw = 2.58e-01, train error = 0.9978
Iter 19, Δw = 2.45e-02, train error = 0.9978
Iter 20, Δw = 1.14e-01, train error = 0.9976
Iter 21, Δw = 4.40e-02, train error = 0.9975
Iter 22, Δw = 9.76e-